In [1]:
# import necessary libraries
from flask import Flask, render_template
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests, pymongo, re, time

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(5)
html = browser.html
soup = bs(html, 'html.parser')

In [3]:
newsinfo = soup.find('div', class_='list_text')
news_title = newsinfo.find('div', class_='content_title').text
news_p = newsinfo.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

Meet the People Behind NASA's InSight Mars Lander
A series of NASA videos highlight scientists and engineers leading the next mission to Mars.


In [4]:
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)
html2 = browser.html
soup2 = bs(html2, 'html.parser')

In [5]:
featured_image_url = soup2.find('article', class_='carousel_item').footer.a.get("data-fancybox-href")
featured_image_url =f'https://www.jpl.nasa.gov{featured_image_url}'
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19977_ip.jpg'

In [6]:
url3 = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url3)
html3 = browser.html
soup3 = bs(html3, 'html.parser')

In [7]:
mars_weather = soup3.find('p', text=re.compile(' hPa, daylight ')).text
mars_weather

'Sol 2108 (2018-07-12), Sunny, high -24C/-11F, low -65C/-84F, pressure at 8.06 hPa, daylight 05:19-17:27'

In [8]:
#This is another way to get the mars_weather, could be better since it filters through two things. 
#first the findall and then filters screen name, then for the text

# paragraph = soup3.find_all('div', {'data-screen-name' : 'MarsWxReport'})
# for x in paragraph:
#     if x(text=re.compile('pressure at')):
#         mars_weather = x(text=re.compile('pressure at'))[0]
#         break
# #         print(x(text=re.compile('pressure at'))[0])
# mars_weather

In [27]:
url4 = 'https://space-facts.com/mars/'
table = pd.read_html(url4)[0].rename(columns={0:"Description",1:"Value"})

html_table = table.to_html(index=False, border=0, classes="table table-hover")
print(html_table)

<table border="1" class="dataframe table table-hover">
  <thead>
    <tr style="text-align: right;">
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [10]:
url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url5)
html5 = browser.html
soup5 = bs(html5, 'html.parser')

In [11]:
imagelist = soup5.find_all('div', class_="description")

hemisphere_image_urls=[]
for x in imagelist:
    titlex= x.h3.text
    urlx = f"https://astrogeology.usgs.gov{x.a['href']}"
    
    browser.visit(urlx)
    soupx = bs(browser.html, 'html.parser')
    
    imagex = soupx.find('img', class_='wide-image')['src']
    imagex = f"https://astrogeology.usgs.gov{imagex}"
    print(imagex + " image url collected")
    
    imgdict = {"title":titlex, "img_url":imagex}
    hemisphere_image_urls.append(imgdict)
    

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg image url collected
https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg image url collected
https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg image url collected
https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg image url collected


In [12]:
hemisphere_image_urls

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [13]:
 MarsInfo = {
        "news_title":news_title,
        "news_p":news_p,
        "featured_image_url":featured_image_url,
        "mars_weather":mars_weather,
        "html_table":html_table,
        "hemisphere_image_urls":hemisphere_image_urls
    }
MarsInfo

{'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19977_ip.jpg',
 'hemisphere_image_urls': [{'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
   'title': 'Cerberus Hemisphere Enhanced'},
  {'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
   'title': 'Schiaparelli Hemisphere Enhanced'},
  {'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
   'title': 'Syrtis Major Hemisphere Enhanced'},
  {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
   'title': 'Valles Marineris Hemisphere Enhanced'}],
 'html_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </t

In [14]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.MarsDB
db.ScrapedCollection.drop()
db.ScrapedCollection.insert_one(MarsInfo)

In [30]:
test = db.ScrapedCollection.find_one()

In [42]:
test["hemisphere_image_urls"][1]

AttributeError: 'dict' object has no attribute 'featured_image_url'